# Fuzzy C-Means Recommender System

## Importing Libraries

Importing necessary libraries for data manipulation, machine learning, and visualization.

In [331]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import os

## Utility Functions

### `load_data()`

Loads the MovieLens 100k dataset and returns both the raw ratings and a user-item pivot table.

**Returns:**
- `ratings`: raw ratings DataFrame with columns `[user_id, item_id, rating, timestamp]`
- `R`: user-item matrix (DataFrame), rows are users, columns are items

In [332]:
def load_data() -> tuple[pd.DataFrame, pd.DataFrame]:
    ratings = pd.read_csv(
        "ml-1m/ratings.dat", 
        sep="::", 
        engine='python', 
        names=["user_id", "item_id", "rating", "timestamp"]
    )
    R = ratings.pivot(index='user_id', columns='item_id', values='rating')
    return ratings, R

### `normalize_user_ratings(R)`

Normalizes each user's ratings by subtracting their mean rating.

**Parameters:**
- `R`: user-item ratings DataFrame (can contain NaN)

**Returns:**
- A DataFrame with normalized ratings, NaNs replaced by 0.

In [333]:
def normalize_user_ratings_simple(R: pd.DataFrame) -> pd.DataFrame:
    """Normalizzazione semplice - sottrazione della media per utente (escludendo NaN)"""
    user_means = R.mean(axis=1)
    R_norm = R.subtract(user_means, axis=0)
    return R_norm.astype(float).fillna(0)

def normalize_zscore_per_user(R: pd.DataFrame) -> pd.DataFrame:
    """Z-score normalizzazione per ogni utente individualmente, con controllo std>0"""
    R_norm = R.copy()
    for user in R_norm.index:
        user_ratings = R_norm.loc[user].dropna()
        if len(user_ratings) > 1 and user_ratings.std() > 0:
            mean_val = user_ratings.mean()
            std_val = user_ratings.std()
            R_norm.loc[user] = (R_norm.loc[user] - mean_val) / std_val
        else:
            # Se std=0 o 1 solo rating, zero-centered
            R_norm.loc[user] = R_norm.loc[user] - user_ratings.mean()
    return R_norm.fillna(0).astype(float)

def split_train_test_per_user(
    R: pd.DataFrame, test_size: float, random_state: int
) -> tuple[pd.DataFrame, pd.DataFrame]:
    R_train = R.copy()
    R_test = pd.DataFrame(index=R.index, columns=R.columns)

    for user in R.index:
        user_ratings = R.loc[user].dropna()
        if len(user_ratings) < 2:
            # Troppo pochi rating per split, tutti in train
            continue

        train_items, test_items = train_test_split(
            user_ratings.index, test_size=test_size, random_state=random_state
        )
        # Imposto test rating su test set, rimuovo da train
        R_test.loc[user, test_items] = R.loc[user, test_items]
        R_train.loc[user, test_items] = np.nan

    return R_train, R_test

def no_user_normalization(R: pd.DataFrame) -> pd.DataFrame:
    """Nessuna normalizzazione per utente, mantieni rating originali"""
    return R.astype(float).fillna(0)

def normalize_minmax_per_user(R: pd.DataFrame) -> pd.DataFrame:
    """Normalizza ogni utente tra 0 e 1"""
    R_norm = R.copy()
    for user in R_norm.index:
        user_ratings = R_norm.loc[user].dropna()
        if len(user_ratings) > 1:
            min_val = user_ratings.min()
            max_val = user_ratings.max()
            if max_val > min_val:
                R_norm.loc[user] = (R_norm.loc[user] - min_val) / (max_val - min_val)
    return R_norm.astype(float).fillna(0)


### `split_train_test_per_user(R, test_size=0.2, random_state=42)`

Splits the user-item matrix into train and test sets, holding out `test_size` ratings per user.

**Parameters:**
- `R`: user-item ratings matrix
- `test_size`: fraction of each user's ratings to hold out for testing
- `random_state`: reproducibility seed

**Returns:**
- `R_train`: training matrix with held-out ratings removed
- `R_test`: test matrix with only the held-out ratings

## Clustering and Prediction Functions

### `fcm_cluster(X, n_clusters=5, m=2.0, max_iter=1000, error=1e-5)`

Performs Fuzzy C-Means clustering on the input matrix `X`.

**Parameters:**
- `X`: scaled user-item matrix (2D array, shape: users × items)
- `n_clusters`: number of fuzzy clusters
- `m`: fuzziness parameter (usually in [1.5, 2.5])
- `max_iter`: maximum number of iterations
- `error`: convergence threshold

**Returns:**
- `cntr`: cluster centroids (shape: n_clusters × features)
- `u`: membership matrix (shape: n_clusters × n_users)

In [334]:
def fcm_cluster(X, n_clusters, m, error, max_iter, seed):
    print("Input data shape:", X.shape)
    print("Input data stats: min =", np.min(X), "max =", np.max(X), "mean =", np.mean(X))
    cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(
        data=X.T, c=n_clusters, m=m, error=error, maxiter=max_iter, seed=seed
    )
    print("Cluster centers shape:", cntr.shape)
    print("Membership matrix shape:", u.shape)
    print("Sample memberships:\n", u[:, :5])
    return cntr, u


### `predict_fcm_soft(cntr, membership)`

Computes soft predictions for each user using fuzzy membership weights and cluster centroids.

**Parameters:**
- `cntr`: cluster centroids (n_clusters × n_items)
- `membership`: user membership matrix (n_clusters × n_users)

**Returns:**
- predicted user-item rating matrix (n_users × n_items)

In [335]:
def predict_fcm_soft(cntr: np.ndarray, membership: np.ndarray) -> np.ndarray:
    n_clusters, n_users = membership.shape
    n_items = cntr.shape[1]
    pred = np.zeros((n_users, n_items))

    for c in range(n_clusters):
        weights = membership[c, :]
        pred += np.outer(weights, cntr[c, :])

    return pred

### `denormalize(pred_norm, R)`

Adds back each user's mean rating to restore original scale.

**Parameters:**
- `pred_norm`: normalized prediction matrix
- `R`: original rating matrix with NaNs

**Returns:**
- prediction matrix on original scale

In [336]:
def denormalize(pred_norm: np.ndarray, R: pd.DataFrame) -> np.ndarray:
    means = R.mean(axis=1).values.reshape(-1, 1)
    return pred_norm + means

## Validation and Visualization Functions

### `evaluate(y_true, y_pred)`

Computes RMSE and MAE between true and predicted ratings.

**Parameters:**
- `y_true`: ground-truth rating matrix (with NaNs)
- `y_pred`: predicted rating matrix

**Returns:**
- RMSE
- MAE

In [337]:
def evaluate(y_true: np.ndarray, y_pred: np.ndarray) -> tuple[float, float]:
    y_true = np.array(y_true, dtype=np.float64)
    y_pred = np.array(y_pred, dtype=np.float64)
    mask = ~np.isnan(y_true)
    mse = mean_squared_error(y_true[mask], y_pred[mask])
    mae = mean_absolute_error(y_true[mask], y_pred[mask])
    return np.sqrt(mse), mae

### `plot_clusters(R_scaled, membership)`

Creates three visualizations of fuzzy clustering results and saves them as PNG files.

**Parameters:**
- `R_scaled`: scaled user-item matrix (numpy array)
- `membership`: fuzzy membership matrix where rows are clusters and columns are users

**Output:**
- `images/fuzzy_clusters_pca.png`: 2D PCA scatter plot of users colored by their dominant cluster
- `images/membership_histogram.png`: histogram showing the distribution of maximum membership values across users
- `images/membership_heatmap.png`: heatmap of membership values for the 10 most uncertain users (highest entropy)

In [338]:
def plot_clusters(R_scaled: np.ndarray, membership: np.ndarray) -> None:
    os.makedirs("images", exist_ok=True)
    
    # PCA to reduce dimensions for visualization
    pca = PCA(n_components=2)
    reduced = pca.fit_transform(R_scaled)
    cluster_labels = np.argmax(membership, axis=0)
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(reduced[:, 0], reduced[:, 1], c=cluster_labels, cmap='Set1', alpha=0.7)
    plt.title("User Clusters (Fuzzy C-Means)")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.colorbar(scatter, label='Cluster Label')
    plt.savefig("images/fuzzy_clusters_pca.png")

    # Histogram of maximum membership values
    plt.figure(figsize=(8, 4))
    max_membership = np.max(membership, axis=0)
    plt.hist(max_membership, bins='auto', color='skyblue', edgecolor='black')
    plt.title("Distribution of Maximum Membership Values")
    plt.xlabel("Maximum Membership Value")
    plt.ylabel("Frequency")
    plt.savefig("images/membership_histogram.png")

    # Heatmap of membership values for the top 10 most uncertain users
    entropy = -np.sum(membership * np.log(membership + 1e-10), axis=0)
    idx_uncertain = np.argsort(entropy)[-10:] 
    plt.figure(figsize=(10, 6))
    sns.heatmap(membership[:, idx_uncertain], cmap='viridis', annot=True)
    plt.title("Heatmap Membership Values for Most Uncertain Users")
    plt.xlabel("User")
    plt.ylabel("Cluster")
    plt.savefig("images/membership_heatmap.png")

In [339]:
def plot_single_normalization(R_norm: np.ndarray, membership: np.ndarray, norm_name: str):
    os.makedirs("images", exist_ok=True)

    pca = PCA(n_components=2)
    reduced = pca.fit_transform(R_norm)
    cluster_labels = np.argmax(membership, axis=0)

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.scatter(reduced[:, 0], reduced[:, 1], c=cluster_labels, cmap='Set1', alpha=0.7)
    plt.title(f"PCA - {norm_name}")
    plt.xlabel("PC1")
    plt.ylabel("PC2")

    plt.subplot(1, 3, 2)
    max_membership = np.max(membership, axis=0)
    plt.hist(max_membership, bins='auto', alpha=0.7)
    plt.title(f"Max Membership - {norm_name}")
    plt.xlabel("Max Membership")
    plt.ylabel("Frequency")

    plt.subplot(1, 3, 3)
    plt.hist(R_norm.flatten(), bins='auto', alpha=0.7)
    plt.title(f"Data Distribution - {norm_name}")
    plt.yscale('log')
    plt.xlabel("Rating Value")
    plt.ylabel("Frequency")

    plt.tight_layout()
    plt.savefig(f"images/comparison_{norm_name}.png", dpi=300, bbox_inches='tight')
    plt.show()


## Main Execution Block

In [340]:
def run_fcm_experiment(R_train_scaled, R_test_scaled, R_train, R_test_aligned, n_clusters, m, max_iter=3000, error=1e-6):
    start_fcm = time.time()
    print("Sample of R_train_scaled:", R_train_scaled[:5, :10])
    print("Min:", np.min(R_train_scaled), "Max:", np.max(R_train_scaled), "Mean:", np.mean(R_train_scaled))
    cntr, u = fcm_cluster(R_train_scaled, n_clusters=n_clusters, m=m, max_iter=max_iter, error=error, seed=31)
    print(f"Membership matrix sample for clusters={n_clusters}, m={m}:\n", u[:, :5])



    u_test, _, _, _, _, _ = fuzz.cluster.cmeans_predict(
        R_test_scaled.T, cntr, m, error=error, maxiter=max_iter)

    pred_train_norm = predict_fcm_soft(cntr, u)
    pred_test_norm = predict_fcm_soft(cntr, u_test)

    pred_train = denormalize(pred_train_norm, R_train)
    pred_test = denormalize(pred_test_norm, R_test_aligned)

    rmse_train, mae_train = evaluate(R_train.values, pred_train)
    rmse_test, mae_test = evaluate(R_test_aligned.values, pred_test)

    avg_max_membership = np.mean(np.max(u, axis=0))
    avg_entropy = np.mean(-np.sum(u * np.log(u + 1e-10), axis=0))

    fcm_time_sec = time.time() - start_fcm

    return {
        "train_rmse": rmse_train,
        "train_mae": mae_train,
        "test_rmse": rmse_test,
        "test_mae": mae_test,
        "avg_max_membership": avg_max_membership,
        "avg_entropy": avg_entropy,
        "fcm_time_sec": fcm_time_sec
    }


def run_experiments_varying_c_m():
    normalizations = {
        "simple_centering": normalize_user_ratings_simple,
        # you can add others here if you want to test normalization too
    }

    results = {}

    ratings, R = load_data()
    R_train, R_test = split_train_test_per_user(R, test_size=0.2, random_state=42)
    R_test_aligned = R_test.reindex(columns=R_train.columns, fill_value=np.nan)

    cluster_values = [4, 6, 8, 10, 12]
    m_values = [1.1, 1.3, 1.5, 1.7, 2.0]

    for norm_name, norm_func in normalizations.items():
        print(f"\n=== Normalization: {norm_name} ===")
        results[norm_name] = {}

        R_train_norm = norm_func(R_train)
        R_test_norm = norm_func(R_test_aligned)

        R_train_scaled = R_train_norm.values
        R_test_scaled = R_test_norm.values
        
        print("R_train_scaled summary:", np.min(R_train_scaled), np.max(R_train_scaled), np.mean(R_train_scaled))
        print("R_test_scaled summary:", np.min(R_test_scaled), np.max(R_test_scaled), np.mean(R_test_scaled))

        for c in cluster_values:
            results[norm_name][c] = {}
            for m in m_values:
                print(f"\nRunning FCM: clusters={c}, m={m}")
                metrics = run_fcm_experiment(
                    R_train_scaled, R_test_scaled,
                    R_train, R_test_aligned,
                    n_clusters=c, m=m
                )
                results[norm_name][c][m] = metrics
                print(f"Train RMSE: {metrics['train_rmse']:.4f}, Test RMSE: {metrics['test_rmse']:.4f}, Avg Max Mem: {metrics['avg_max_membership']:.4f}, Avg Entropy: {metrics['avg_entropy']:.4f}")

    # Save results for further analysis
    os.makedirs("results", exist_ok=True)
    with open("results/fcm_experiments_c_m.json", "w") as f:
        json.dump(results, f, indent=4)

    return results


# Run this to perform your experiments:
results_grid = run_experiments_varying_c_m()


=== Normalization: simple_centering ===
R_train_scaled summary: -3.878787878787879 3.690909090909091 1.5315993306194372e-19
R_test_scaled summary: -3.825 3.7142857142857144 5.268542982374323e-19

Running FCM: clusters=4, m=1.1
Input data shape: (6040, 3706)
Input data stats: min = -3.878787878787879 max = 3.690909090909091 mean = 1.5315993306194372e-19
Cluster centers shape: (4, 3706)
Membership matrix shape: (4, 6040)
Sample memberships:
 [[0.25       0.25       0.25       0.25       0.25      ]
 [0.25000004 0.25       0.25000002 0.25000005 0.25      ]
 [0.25000004 0.25       0.25000002 0.25000005 0.25      ]
 [0.24999991 0.25       0.24999995 0.24999991 0.25      ]]
Membership matrix sample for clusters=4, m=1.1:
 [[0.25       0.25       0.25       0.25       0.25      ]
 [0.25000004 0.25       0.25000002 0.25000005 0.25      ]
 [0.25000004 0.25       0.25000002 0.25000005 0.25      ]
 [0.24999991 0.25       0.24999995 0.24999991 0.25      ]]
Train RMSE: 1.0097, Test RMSE: 0.9993, A